Importing datasets into pandas

In [1]:
import matplotlib as plt
import kaggle
import zipfile
import pandas as pd
import numpy as np
import os
import pickle


In [2]:
#downloading pollution_us dataset if doesnt exist locally
us_pollution_filename = 'pollution_us_2000_2016.csv'
if not(os.path.isfile(us_pollution_filename)):
    kaggle.api.dataset_download_file('sogun3/uspollution', us_pollution_filename)
    zip_ref = zipfile.ZipFile(us_pollution_filename+'.zip', 'r')
    zip_ref.extractall()
    zip_ref.close()
df_us_pollution = pd.read_csv(us_pollution_filename)

In [3]:
#assuming resp dataset already exists locally
resp_data_path = 'IHME_USA_COUNTY_RESP_DISEASE_MORTALITY_1980_2014_NATIONAL_Y2017M09D26.XLSX'
list_sheet_names = ['Chronic respiratory diseases', 'Chronic obstructive pulmonary ', 'Pneumoconiosis', 'Silicosis',
                   'Asbestosis', 'Coal workers pneumoconiosis', 'Other pneumoconiosis', 'Asthma', 'Interstitial lung disease',
                   'Other chronic respiratory ']
df_resp_disease = pd.read_excel(resp_data_path, skiprows=1, sheet_name=list_sheet_names)


In [4]:
for x in list_sheet_names:
    #dropping mortality rate measured below year 2000
    #dropping any na values
    df_resp_disease[x].drop(['Mortality Rate, 1980*', 'Mortality Rate, 1985*', 'Mortality Rate, 1990*', 'Mortality Rate, 1995*'],
                           inplace=True, errors='ignore', axis=1)
    df_resp_disease[x].dropna(inplace=True)
    #dropping county data, state mortality rate is average of all counties
    df_resp_disease[x].drop(df_resp_disease[x][df_resp_disease[x].FIPS>57].index, inplace=True)
#     #Extracting state out of Location
#     df_resp_disease[x]['state'] = df_resp_disease[x]['Location']\
#     .apply(lambda x: x.split(',')[1].strip() if len(x.split(',')) > 1 else x.strip())
    df_resp_disease[x]['mortality_2000'] = df_resp_disease[x]['Mortality Rate, 2000*']\
    .apply(lambda x: x.split(' ')[0].strip())
    df_resp_disease[x]['mortality_2005'] = df_resp_disease[x]['Mortality Rate, 2005*']\
    .apply(lambda x: x.split(' ')[0].strip())
    df_resp_disease[x]['mortality_2010'] = df_resp_disease[x]['Mortality Rate, 2010*']\
    .apply(lambda x: x.split(' ')[0].strip())
    df_resp_disease[x]['mortality_2014'] = df_resp_disease[x]['Mortality Rate, 2014*']\
    .apply(lambda x: x.split(' ')[0].strip())
    df_resp_disease[x].drop(['Mortality Rate, 2000*', 'Mortality Rate, 2005*', \
                             'Mortality Rate, 2010*', 'Mortality Rate, 2014*', \
                            '% Change in Mortality Rate, 1980-2014'], axis=1, inplace=True)


In [5]:
#stacking all sheets into one dataframe vertically
df_resp_mortality_cleaned = pd.DataFrame()
for x in list_sheet_names:
    for column in ['mortality_2000','mortality_2005','mortality_2010','mortality_2014']:
        df_tmp = pd.DataFrame(df_resp_disease[x][column]).rename(columns={column : 'mortality_rate'})
        df_tmp['year']=int(column[-4:])
        df_tmp['disease']=x
        df_tmp['state']=df_resp_disease[x].Location
        df_resp_mortality_cleaned = df_resp_mortality_cleaned.append(df_tmp)

df_resp_mortality_cleaned.reset_index(inplace=True, drop=True)
pickle.dump( df_resp_mortality_cleaned, open( 'df_resp_mortality.pkl', "wb" ) )

#stacking all sheets into one dataframe horizontally
df_resp_mortality_horizontal_stack = pd.DataFrame()
for column in ['mortality_2000', 'mortality_2005', 'mortality_2010', 'mortality_2014']:
    df_tmp = pd.DataFrame()
    df_tmp['state']=df_resp_disease[x].Location
    for x in list_sheet_names:
        df_tmp[x] = df_resp_disease[x][column].astype(float)
        df_tmp['year']=int(column[-4:])

    df_resp_mortality_horizontal_stack = df_resp_mortality_horizontal_stack.append(df_tmp)

df_resp_mortality_horizontal_stack.reset_index(inplace=True, drop=True)
pickle.dump( df_resp_mortality_horizontal_stack, open( 'df_resp_mortality_horizontal_stack.pkl', "wb" ) )


In [6]:
#df_us_pollution does not have data for all counties, therefore need to do it based on state
#need to aggregate by year in date local field to match with mortality data ([2000, 2005, 2010, 2014])
df_us_pollution['year'] = pd.DatetimeIndex(df_us_pollution['Date Local']).year
series_year_data = df_us_pollution['year']
df_us_pollution['year_bin'] = np.where(series_year_data<=2000, 2000,
                                      np.where(np.logical_and(series_year_data>2000, series_year_data<=2005), 2005,
                                      np.where(np.logical_and(series_year_data>2005, series_year_data<=2010), 2010, 
                                      np.where(np.logical_and(series_year_data>2010, series_year_data<=2014), 2014, np.nan))))
df_us_pollution_cleaned = df_us_pollution.drop(['State Code', 'County Code', 'Site Num', 'Address', 'County', 'City', 
                                                'Date Local'], axis=1)
pickle.dump( df_us_pollution_cleaned, open( 'df_us_pollution.pkl', "wb" ) )


In [7]:
#merging pollutio data and respiratory mortality rate data
df_merged = df_us_pollution_cleaned.merge(df_resp_mortality_horizontal_stack, left_on=['State','year_bin'], right_on=['state','year'])
df_merged_tmp = df_merged.drop(['Unnamed: 0', 'state', 'year_y', 'year_x',  'NO2 Units', 
                                'O3 Units', 'O3 1st Max Value', 'NO2 1st Max Value', 'O3 1st Max Value'
                                'NO2 1st Max Value','NO2 1st Max Hour', 'NO2 AQI', 'O3 Units'
                               'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', 'SO2 Units',
                                'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI',
                               'CO Units', 'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI'],axis=1, errors='ignore')

In [9]:
#calculating corr coefficient
correlation_graph = df_merged_tmp.corr()
#transforming the result table to more readable format
correlation_graph[:4][list_sheet_names].T
pickle.dump( correlation_graph, open( 'correlation_table.pkl', "wb" ) )

In [12]:
#grouping df_merged_tmp by state and year for better readability
df_grouped = df_merged_tmp.groupby(['State','year_bin']).mean()
pickle.dump( df_grouped, open( 'df_grouped.pkl', "wb" ) )

In [14]:
df_grouped

NO2 Mean   O3 Mean  SO2 Mean   CO Mean  \
State          year_bin                                            
Alabama        2014.0     9.659176  0.023489  1.145848  0.206396   
Alaska         2014.0    11.658510  0.010307  5.951531  0.455444   
Arizona        2000.0    26.487502  0.023655  2.303226  0.745756   
               2005.0    22.094448  0.022779  1.770712  0.622628   
               2010.0    18.368101  0.025264  1.385875  0.458466   
               2014.0    16.491932  0.026972  0.832914  0.353505   
Arkansas       2010.0     9.846413  0.026378  1.691583  0.467806   
               2014.0     9.801567  0.026194  1.277770  0.411502   
California     2000.0    17.580989  0.023675  1.721004  0.644488   
               2005.0    15.934494  0.025593  1.680635  0.557800   
               2010.0    13.039071  0.026380  1.088312  0.398237   
               2014.0    11.525273  0.026494  0.571274  0.349698   
Colorado       2000.0    14.902469  0.017066  2.138058  0.611388   
               2005.0    22.754396  0.018434  2.071892  0.589139   
               2010.0    21.190122  0.022836  1.794106  0.487132   
               2014.0    16.852723  0.027600  1.114965  0.327464   
Connecticut    2010.0    11.237516  0.026475  1.425210  0.320051   
               2014.0     7.640676  0.030135  0.676703  0.210676   
Delaware       2014.0    11.561120  0.026361  1.078697  0.253042   
Florida        2000.0    12.453038  0.026261  1.973912  0.725072   
               2005.0    10.553728  0.024742  0.724882  0.526602   
               2010.0     6.167319  0.028469  0.438926  0.307695   
               2014.0     5.077699  0.026778  0.060314  0.393216   
Georgia        2010.0    16.500656  0.016584  1.147945  0.396889   
               2014.0    11.232157  0.021218  0.506432  0.318709   
Hawaii         2014.0     3.014657  0.025682  0.960046  0.359040   
Idaho          2010.0     8.875329  0.027761  0.305804  0.173158   
               2014.0     9.925650  0.012999  0.715267  0.272311   
Illinois       2000.0    26.437457  0.017105  6.064269  0.702893   
               2005.0    20.075347  0.021679  4.743110  0.551736   
...                            ...       ...       ...       ...   
Ohio           2010.0     4.710887  0.031634  2.284301  0.331439   
               2014.0    13.226551  0.023881  3.038836  0.261785   
Oklahoma       2000.0     4.770656  0.037325  0.852076  0.047977   
               2005.0     6.911370  0.033445  0.895476  0.032594   
               2010.0     6.930125  0.027996  0.829966  0.191770   
               2014.0     6.518085  0.032371  0.478650  0.183807   
Oregon         2010.0    10.331479  0.018858  1.285621  0.328589   
               2014.0     9.120014  0.019535  0.824220  0.291483   
Pennsylvania   2000.0    16.130558  0.024541  5.154716  0.302550   
               2005.0    14.326985  0.026598  5.065616  0.276917   
               2010.0    11.613775  0.027467  4.327924  0.226610   
               2014.0    10.374058  0.024994  2.163690  0.203221   
Rhode Island   2014.0     6.919726  0.030120  0.594575  0.220493   
South Carolina 2005.0     2.355526  0.032039  0.700837  0.260566   
               2010.0     1.765775  0.031686  0.846189  0.113951   
South Dakota   2014.0     5.133461  0.031070  0.495170  0.179779   
Tennessee      2005.0     3.464797  0.033944  1.245587  1.000000   
               2010.0     0.879084  0.040502  0.856434  0.175269   
Texas          2000.0    16.346748  0.024590  2.726069  0.436668   
               2005.0    15.043758  0.026615  1.758877  0.369800   
               2010.0    11.104204  0.024737  1.003921  0.226197   
               2014.0     9.915926  0.026569  0.466841  0.183004   
Utah           2014.0    12.473583  0.032862  0.419309  0.351378   
Virginia       2000.0    17.915559  0.022609  9.042594  0.644428   
               2010.0    11.608953  0.027814  3.302080  0.384281   
               2014.0     7.069008  0.030113  0.571376  0.266226   
Washington    

In [15]:
df_grouped.corr()[:4][list_sheet_names].T

,NO2 Mean,O3 Mean,SO2 Mean,CO Mean
Chronic respiratory diseases,-0.247818,0.308144,-0.128498,-0.133533
Chronic obstructive pulmonary,-0.228727,0.314149,-0.130729,-0.127888
Pneumoconiosis,-0.012178,0.076642,0.293256,-0.106835
Silicosis,0.237709,0.006656,0.138545,0.149144
Asbestosis,-0.113623,-0.098369,-0.039430,-0.103005
Coal workers pneumoconiosis,0.021966,0.091242,0.315665,-0.094508
Other pneumoconiosis,-0.057633,0.158370,0.184910,-0.034293
Asthma,0.377504,-0.093807,0.514091,0.508328
Interstitial lung disease,-0.421299,0.057742,-0.310984,-0.246545
Other chronic respiratory,-0.423651,0.160155,-0.219376,-0.283252
